In [2]:
%env THEANO_FLAGS=device=gpu2
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

env: THEANO_FLAGS=device=gpu2


In [6]:
from utils.southpark import get_conversations
conversations = get_conversations("/srv/hd7/jheuristic/cartman/sp.json")
    
from utils.preprocessor import Preprocessor

preproc = Preprocessor.from_conversations(conversations,verbose=True)


100%|██████████| 7285/7285 [00:03<00:00, 1936.65it/s]

23281 out of 23281 tokens, coverage=1.00000)


# model

In [7]:
from models.twoline_model import ConversationModel
model = ConversationModel(preproc,bottleneck_size=512)

/home/jheuristic/thenv/local/lib/python2.7/site-packages/agentnet/agent/recurrence.py:228: UserWarning: You are giving Recurrence an input sequence of undefined length (None).
Make sure it is always above <unspecified>(n_steps) you specified for recurrence
  "Make sure it is always above {}(n_steps) you specified for recurrence".format(n_steps or "<unspecified>"))
/home/jheuristic/thenv/local/lib/python2.7/site-packages/agentnet/agent/recurrence.py:453: UserWarning: Warning: recurrent loop without unroll_scan got nonempty random state updates list. That happened because there is some source of randomness (e.g. dropout) inside recurrent step graph. To compile such graph, one must either call .get_automatic_updates() right after .get_output and pass these updates to a function, or use no_defalt_updates=True when compiling theano.function.
  warn("Warning: recurrent loop without unroll_scan got nonempty random state updates list. That happened"


# training

In [8]:
max_len=10
batch_size=100
n_epochs=100

In [9]:
from utils.generate import get_phrase_pairs,iterate_minibatches

convs_ix = list(preproc.preprocess_conversations(conversations,verbose=True,max_len=max_len))

prev_phrases,reference_answers = get_phrase_pairs(convs_ix,
                                                  speaker_filter = lambda s1,s2: 'cartman' in s2).swapaxes(0,1).astype('int32')




100%|██████████| 7285/7285 [00:04<00:00, 1639.32it/s]


In [10]:
epoch_counter = 1
ce = []

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
for _ in range(n_epochs):
    for b_x,b_y in iterate_minibatches([prev_phrases,reference_answers],batch_size,shuffle=True):
        ce.append(model.train_fun(b_x,b_y))
        
        epoch_counter +=1
        
        if epoch_counter %25==0:
            print epoch_counter,'iterations...'
    print "beginning new loop..."
    plt.plot(ce)
    plt.show()
    
    ph =preproc.ix_to_phrase(b_x[0])
    print 'A:', ph
    print 'B:', model.reply(ph)
    print 'B true:',preproc.ix_to_phrase(b_y[0])
        

In [15]:
from agentnet.utils.persistence import save
save(model.decoder_training_rec,"/srv/hd7/jheuristic/agentnet_snapshots/southpark_simple_cm_cartman_epoch%i.pcl"%epoch_counter)

# apply

In [11]:
from agentnet.utils.persistence import load
load(model.decoder_training_rec,"/srv/hd7/jheuristic/agentnet_snapshots/southpark_simple_cm_cartman_epoch15484.pcl")

In [69]:
model.greed.set_value(1.5)

Q = ["Who's Cartman?",
     "Who are you?",
     "what's up?",
     "Are you serious?",
     "What for?",
     "Do you know him?",
     "No!",
     "Fuck you!",
     "What do you think?",
     "What a mess!",
     "Are you gay?",
     "They killed kenny!",
     "Jesus!"
    ]
for q in Q:
    print '>>',q
    print model.reply(q)

>> Who's Cartman?
beth ! my god ! go !
>> Who are you?
i am the dawg , the big bad dawg
>> what's up?
this is a jew , kenny ! my
>> Are you serious?
it look . get it from here
>> What for?
i don't know . there's my dad !
>> Do you know him?
it looks like kyle . i need a
>> No!
a mountain lion ! okay
>> Fuck you!
yaaaaah ! it's hard , kyle ! !
>> What do you think?
shooting a d - good . . .
>> What a mess!
i am a gay of my child .
>> Are you gay?
it works . get my back to
>> They killed kenny!
they come ! my
>> Jesus!
uh - huh . .
